In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score


In [24]:
# Load in your data here
data = pd.read_csv("IQmoodhc_deltaPSD.csv") #file name here


In [25]:
# Data preprocessing
X = data.drop('Class', axis=1)
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [26]:
# PCA data
from sklearn.decomposition import PCA
pca = PCA(0.95)
X_pca = pca.fit_transform(X)
X_pca.shape
X = X.dropna()

X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.1, random_state=30)

# SVM Values

In [27]:
svclassifier = SVC(kernel='linear',probability=True)
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)
print('SVM accuracy:', svclassifier.score(X_test, y_test))
print('SVM classification report:\n', classification_report(y_test, y_pred))
y_scores = svclassifier.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
# Calculate AUC
roc_auc = auc(fpr, tpr)
print('SVM AUC value:', roc_auc)
newsvm=SVC(kernel='linear', probability=True)
newsvm.fit(X_train_pca, y_train)
y_pred = newsvm.predict(X_test_pca)
print('\nSVM accuracy for PCA:', newsvm.score(X_test_pca, y_test))
print('SVM classification report for PCA:\n', classification_report(y_test, y_pred))
y_scores = newsvm.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('SVM AUC value for PCA:', roc_auc)


SVM accuracy: 0.8055555555555556
SVM classification report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.81      1.00      0.89        29

    accuracy                           0.81        36
   macro avg       0.40      0.50      0.45        36
weighted avg       0.65      0.81      0.72        36

SVM AUC value: 0.6206896551724138


/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



SVM accuracy for PCA: 0.8611111111111112
SVM classification report for PCA:
               precision    recall  f1-score   support

           0       0.67      0.57      0.62         7
           1       0.90      0.93      0.92        29

    accuracy                           0.86        36
   macro avg       0.78      0.75      0.77        36
weighted avg       0.85      0.86      0.86        36

SVM AUC value for PCA: 0.8866995073891626


# KNN Values

In [28]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
print('KNN accuracy:', knn.score(X_test, y_test))
y_pred = knn.predict(X_test)
print('KNN classification report:\n',classification_report(y_test, y_pred))
y_scores = knn.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('KNN AUC value:', roc_auc)
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_pca, y_train)
print('\nKNN accuracy for PCA:', knn.score(X_test_pca, y_test))
y_pred = knn.predict(X_test_pca)
print('KNN classification report for PCA\n', classification_report(y_test, y_pred))
y_scores = knn.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('KNN AUC value for PCA',roc_auc)

KNN accuracy: 0.6944444444444444
KNN classification report:
               precision    recall  f1-score   support

           0       0.17      0.14      0.15         7
           1       0.80      0.83      0.81        29

    accuracy                           0.69        36
   macro avg       0.48      0.49      0.48        36
weighted avg       0.68      0.69      0.69        36

KNN AUC value: 0.4359605911330049

KNN accuracy for PCA: 0.8055555555555556
KNN classification report for PCA
               precision    recall  f1-score   support

           0       0.50      0.57      0.53         7
           1       0.89      0.86      0.88        29

    accuracy                           0.81        36
   macro avg       0.70      0.72      0.71        36
weighted avg       0.82      0.81      0.81        36

KNN AUC value for PCA 0.7783251231527094


# Random Forest Values

In [29]:
model = RandomForestClassifier(n_estimators=40)
model.fit(X_train, y_train)
print('Random Forest accuracy:', model.score(X_test, y_test))
y_pred = model.predict(X_test)
print('Random Forest classification report\n',classification_report(y_test, y_pred))
y_scores = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('Random Forest AUC value:', roc_auc)
pmodel = RandomForestClassifier(n_estimators=40)
pmodel.fit(X_train_pca, y_train)
print('\nRandom Forest accuracy for PCA:', pmodel.score(X_test_pca, y_test))
y_pred = pmodel.predict(X_test_pca)
print('Random Forest classification report for PCA:\n',classification_report(y_test, y_pred))
y_scores = pmodel.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('Random Forest AUC value for PCA:', roc_auc)

Random Forest accuracy: 0.6666666666666666
Random Forest classification report
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.77      0.83      0.80        29

    accuracy                           0.67        36
   macro avg       0.39      0.41      0.40        36
weighted avg       0.62      0.67      0.64        36

Random Forest AUC value: 0.3497536945812808

Random Forest accuracy for PCA: 0.8055555555555556
Random Forest classification report for PCA:
               precision    recall  f1-score   support

           0       0.50      0.43      0.46         7
           1       0.87      0.90      0.88        29

    accuracy                           0.81        36
   macro avg       0.68      0.66      0.67        36
weighted avg       0.80      0.81      0.80        36

Random Forest AUC value for PCA: 0.7044334975369458


In [12]:
rfc_search_space = {
    'n_estimators': range(10, 101),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 51),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}


In [13]:
rfc = RandomForestClassifier()

In [14]:
random_search = RandomizedSearchCV(estimator=rfc, param_distributions=rfc_search_space, n_iter=100, cv=5)
random_search.fit(X_train, y_train)

,estimator,RandomForestClassifier()
,param_distributions,"{'bootstrap': [True, False], 'criterion': ['gini', 'entropy'], 'max_depth': range(2, 51), 'max_features': ['sqrt', 'log2', ...], ...}"
,n_iter,100
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [15]:
best_params = random_search.best_params_
rfc = RandomForestClassifier(**best_params)

In [16]:
rfc.fit(X_train, y_train)


,n_estimators,97
,criterion,'gini'
,max_depth,44
,min_samples_split,3
,min_samples_leaf,7
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [17]:
y_pred = rfc.predict(X_test)


In [18]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8055555555555556


# Logistic Regression values

In [19]:
model = LogisticRegression()
model.fit(X_train, y_train)
print('Logistic Regression accuracy:', model.score(X_test, y_test))
y_pred = model.predict(X_test)
print('Logistic Regression classification report:\n',classification_report(y_test, y_pred))
y_scores = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('Logistic Regression AUC value:',roc_auc)
pmodel = LogisticRegression()
pmodel.fit(X_train_pca, y_train)
print('\nLogistic Regression accuracy for PCA:',pmodel.score(X_test_pca, y_test))
y_pred = pmodel.predict(X_test_pca)
print('Logistic Regression classification report for PCA:\n',classification_report(y_test, y_pred))
y_scores = pmodel.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('Logistic Regression AUC value for PCA',roc_auc)

Logistic Regression accuracy: 0.7638888888888888
Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.80      0.95      0.87        58

    accuracy                           0.76        72
   macro avg       0.40      0.47      0.43        72
weighted avg       0.64      0.76      0.70        72

Logistic Regression AUC value: 0.43103448275862066

Logistic Regression accuracy for PCA: 0.7916666666666666
Logistic Regression classification report for PCA:
               precision    recall  f1-score   support

           0       0.47      0.50      0.48        14
           1       0.88      0.86      0.87        58

    accuracy                           0.79        72
   macro avg       0.67      0.68      0.68        72
weighted avg       0.80      0.79      0.79        72

Logistic Regression AUC value for PCA 0.8399014778325123


/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# XGB Values

In [20]:
clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
y_pred = clf.predict(X_test)
print('XGB accuracy:',clf.score(X_test, y_test))
print('XGB Classification report:\n',classification_report(y_test, y_pred))
y_scores = clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('XGB AUC value:',roc_auc)
pxgb = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
pxgb.fit(X_train_pca, y_train, eval_set=[(X_test_pca, y_test)])
print('\nXGB accuracy for PCA:', pxgb.score(X_test_pca, y_test))
y_pred = pxgb.predict(X_test_pca)
print('XGB classification report for PCA:\n',classification_report(y_test, y_pred))
y_scores = pxgb.predict_proba(X_test_pca)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)
print('XBG AUC for PCA:', roc_auc)

[0]	validation_0-logloss:0.53702
[1]	validation_0-logloss:0.52499
[2]	validation_0-logloss:0.55538
XGB accuracy: 0.7638888888888888
XGB Classification report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.80      0.95      0.87        58

    accuracy                           0.76        72
   macro avg       0.40      0.47      0.43        72
weighted avg       0.64      0.76      0.70        72

XGB AUC value: 0.5277093596059114
[0]	validation_0-logloss:0.46295
[1]	validation_0-logloss:0.45048
[2]	validation_0-logloss:0.45069

XGB accuracy for PCA: 0.8472222222222222
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.71      0.36      0.48        14
           1       0.86      0.97      0.91        58

    accuracy                           0.85        72
   macro avg       0.79      0.66      0.69        72
weighted avg       0.83      0

# Elastic Net Values

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
elastic_net = ElasticNet()
elastic_net.fit(X_train, y_train)
y_pred = elastic_net.predict(X_test)
print('Mean absolute error:',mean_absolute_error(y_test, y_pred))
print('Mean squared error:', mean_squared_error(y_test, y_pred))
print('r2 score:', r2_score(y_test, y_pred))
pcaX_train = scaler.fit_transform(X_train_pca)
pcaX_test = scaler.fit_transform(X_test_pca)
elastic_net = ElasticNet()
elastic_net.fit(pcaX_train, y_train)
y_pred = elastic_net.predict(pcaX_test)
print('\nMean absolute error for PCA:',mean_absolute_error(y_test, y_pred))
print('Mean squared error for PCA:', mean_squared_error(y_test, y_pred))
print('r2 score for PCA:', r2_score(y_test, y_pred))

Mean absolute error: 0.3669183120402633
Mean squared error: 0.16434210011587422
r2 score: -0.04919882635553208

Mean absolute error for PCA: 0.3669183120402633
Mean squared error for PCA: 0.16434210011587422
r2 score for PCA: -0.04919882635553208
